In [2]:
import random
import os
import re
import numpy as np
from pprint import pprint
print(os.getcwd())
from Backend.py_utils import *
from Backend import perf_calc as perf_calc
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from Backend import perf_calc as perf_calc
from Backend import rotation_calc as rotation_calc
from Backend import refusal as refusal
from database import database_handler
from fastapi.responses import JSONResponse
from Backend import handle_route as handle_route
from pydantic import BaseModel 
from Backend import wind_calc
import sqlite3

g:\CompSciProject
g:\CompSciProject
g:\CompSciProject
g:\CompSciProject


In [11]:
def search_airport(partial_name):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    words = partial_name.split()
    conditions = " AND ".join(["name LIKE ? COLLATE NOCASE" for _ in words])

    query = f"""
    SELECT name, icao,type
    FROM airports 
    WHERE {conditions} 
    ORDER BY 
        LENGTH(name), 
        INSTR(LOWER(name), LOWER(?)) 
    """

    params = tuple(f"%{word}%" for word in words) + (words[0],)

    cursor.execute(query, params)
    results = cursor.fetchall()
    connect_to_db.close()

    return results
search_airport("hello")

[('Othello Municipal Airport', 'KS70', 'small_airport')]

In [11]:
from math import cos, asin, sqrt, pi
import sqlite3
def nearby_points(point_lat, point_lng, area_code):
    database_path = "database/nav_data.db"
    connect_to_db = sqlite3.connect(database_path)
    cursor = connect_to_db.cursor()

    cursor.execute("SELECT waypoint_identifier, waypoint_latitude, waypoint_longitude FROM waypoints WHERE area_code = ?", (area_code,))

    data = cursor.fetchall()

    def distance(lat1, lon1, lat2, lon2):
        r = 6371 # km
        p = pi / 180

        a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
        return 2 * r * asin(sqrt(a))
    
    for waypoint in data:
        waypoint_id, waypoint_lat, waypoint_lng = waypoint
        dist = distance(point_lat, point_lng, waypoint_lat, waypoint_lng)
        print(f"Distance to waypoint {waypoint_id}: {dist} km")

nearby_points(45.11416667,0.61805556, "EUR")

Distance to waypoint 31SF: 3578.899958678808 km
Distance to waypoint 5945N: 3392.6460598691215 km
Distance to waypoint 5950N: 3678.129701396193 km
Distance to waypoint 6045N: 3407.0065461388167 km
Distance to waypoint 6050N: 3683.4641939910593 km
Distance to waypoint 6061N: 4294.204115502877 km
Distance to waypoint 6145N: 3424.5751006474334 km
Distance to waypoint 6150N: 3691.7609937382754 km
Distance to waypoint 6245N: 3445.302233820312 km
Distance to waypoint 6250N: 3702.999954220758 km
Distance to waypoint 6340N: 3226.486670439507 km
Distance to waypoint 6345N: 3469.130843230878 km
Distance to waypoint 6350N: 3717.1540703866067 km
Distance to waypoint 6355N: 3968.262371891656 km
Distance to waypoint 6440N: 3263.714076452078 km
Distance to waypoint 6445N: 3495.9969495221576 km
Distance to waypoint 6450N: 3734.1897930679356 km
Distance to waypoint 6455N: 3975.9762912705773 km
Distance to waypoint 6540N: 3303.972925872866 km
Distance to waypoint 6545N: 3525.830484711855 km
Distance to 